In [27]:
import numpy as np
from ISMgas.SupportingFunctions import load_pickle
from ISMgas.globalVars import c_kms
from astropy.table import Table
import os

In [16]:
lensIDS  = ['CSWA38', 'CSWA103', 'CSWA128', 'CSWA164', 'CSWA19', 'CSWA2', 'CSWA40', '8oclock', 'clone', 'AGEL183520', 'AGEL014106', 'AGEL231935', 'RCSGA0327-G', 'J1527', 'J1458', 'J1429', 'J0108', 'J0004', 'cosmiceye', 'horseshoe']

## Build database from stored fits

In [17]:
db = {} ## database
def getQuantity(qty, db=db, fit='DG'):
    return(np.array([getNestedArrayValue(db, [i,fit,qty]) for i in lensIDS]))

def getQuantityNew(qty,  db=db):
    return(np.array([getNestedArrayValue(db, [i] + qty) for i in lensIDS]))

def getProp(qty):
    return(np.array([getNestedArrayValue(db, [i,'prop',qty]) for i in lensIDS]))

for idx in lensIDS:
    db[idx] =  load_pickle("../data/2022_ISMkinematics/%s/%s-smoothedSpectra.pkls"%(idx,idx))
 
    db[idx]['prop'] = dbLenses[idx]
    db[idx]['raw']  = load_pickle("../data/2022_ISMkinematics/%s/%s-combined.pkls"%(idx,idx))
    db[idx]['DG']   = load_pickle("../data/2022_ISMkinematics/%s/%s-DG-Results.pkl"%(idx,idx))
    db[idx]['SG']   = load_pickle("../data/2022_ISMkinematics/%s/%s-SG-Results.pkl"%(idx,idx))    
         
    foo_dummywav = np.arange(-1300,1300,1)
    foo_dummyflux_cf = np.array([
        db[idx]['DG']['A_out']*np.exp(-0.5*(db[idx]['DG']['v_out'] -i)**2/db[idx]['DG']['sig_out']**2) +
        db[idx]['DG']['A1_out']*np.exp(-0.5*(db[idx]['DG']['v1_out'] -i)**2/db[idx]['DG']['sig1_out']**2) 
        for i in foo_dummywav
    ])
    
    db[idx]['DG']['fitted_wav']  = foo_dummywav
    db[idx]['DG']['fitted_flux']  = foo_dummyflux_cf
    
    db[idx]['DG']['cf_max'] = np.max(foo_dummyflux_cf)
    db[idx]['DG']['area']   = np.sum(foo_dummyflux_cf)
    
    ## EW (in A) -- see notes above
    db[idx]['DG']['EW']     = np.sum(foo_dummyflux_cf)*(1*2796)/c_kms 
    
    ## Skewness 
    db[idx]['DG']['|v50-v95|']    = np.abs(db[idx]['DG']['v50'] - db[idx]['DG']['v05'])
    db[idx]['DG']['|v50-v05|']    = np.abs(db[idx]['DG']['v50'] - db[idx]['DG']['v95'])
    db[idx]['DG']['skewness']     = db[idx]['DG']['|v50-v95|']/db[idx]['DG']['|v50-v05|']  - 1 

    
    db[idx]['SG']['|v50-v95|']    = np.abs(db[idx]['SG']['v50'] - db[idx]['SG']['v05'])
    db[idx]['SG']['|v50-v05|']    = np.abs(db[idx]['SG']['v50'] - db[idx]['SG']['v95'])
    db[idx]['SG']['skewness']     = db[idx]['SG']['|v50-v95|']/db[idx]['SG']['|v50-v05|']  - 1   
    
    ### Storing the first and second fit component separately ###
    
    foo_dummywav = np.arange(-1300,1300,1)
    foo_dummyflux_cf = np.array([
        db[idx]['DG']['A_out']*np.exp(-0.5*(db[idx]['DG']['v_out'] -i)**2/db[idx]['DG']['sig_out']**2)  for i in foo_dummywav
    ])
    db[idx]['DG']['fitted_wav_comp1']  = foo_dummywav
    db[idx]['DG']['fitted_flux_comp1']  = foo_dummyflux_cf    
    
    
    foo_dummywav = np.arange(-1300,1300,1)
    foo_dummyflux_cf = np.array([
        db[idx]['DG']['A1_out']*np.exp(-0.5*(db[idx]['DG']['v1_out'] -i)**2/db[idx]['DG']['sig1_out']**2) 
        for i in foo_dummywav
    ])
    db[idx]['DG']['fitted_wav_comp2']  = foo_dummywav
    db[idx]['DG']['fitted_flux_comp2']  = foo_dummyflux_cf    



## Store all spectra in files

In [29]:
saveDir = "../data/2022_ISMkinematics_csv"
for idx in lensIDS:
    raw = db[idx]['raw']
    linenames = raw['line_name']
    for count,linename in enumerate(linenames):
        spec = raw['spectra_interp'][count]
        wav = raw['wav'][count]
        sigma = raw['sigma'][count]
        
        t = Table(
            data = [
                wav, spec, sigma
            ],
            names = ['Velocity(km/s)','Flux(Normalized)','Error(1sigma)']
        )
        try:
            t.write(f"{saveDir}/{linename}/{idx}.csv")
        except FileNotFoundError:
            os.mkdir(f"{saveDir}/{linename}")
            t.write(f"{saveDir}/{linename}/{idx}.csv")

        
    